# 1. Solution Overview

## 1.1. Data Preprocessing / Feature Engineering
In general, our solution does not use complex feature engineering.

And, we use the following data processing methods to improve the stability of training and the final performance: 
1) Normalize the noisy seismic and denoised seismic on the entire volume to 0 - 255 without clip;
2) Extract 2.5D slices from 3D volumes for training and test. And use the entire slice (1259x300) for training and testing;
3) Use horizontal/vertical flip data augmentation during training and horizontal/vertical flip test time augmentation during testing;

<font color=red>Note: In addition to some regular data enhancement, I used 2.5D technology, which greatly improved my score. Because the textures between adjacent slices are similar, several adjacent slices are put together as the input of the model, and the label of the middle slice is used to supervise the model. This will provide more auxiliary information to the model and help improve performance. However, it should be noted that using too many adjacent slices will have a negative impact, because the difference between slices that are too far apart is too large, which will bring a lot of noise in training.</font>

<font color=red>After experiments, I found that the performance is best when the number of adjacent slices set to 3. Please refer to the pipeline below for detailed training and inference processes.</font>


![Training Process](images/training_process.png "Training Process")


![Inference Process](images/Inference_process.png "Inference Process")

## 1.2. Model description

As for the model details, we use Unet as our final model, with efficientnet_v2_s as backbone. 

And we use the [timm pretrained model](https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-effv2-weights/tf_efficientnetv2_s_21k-6337ad01.pth) on [Imagenet dataset](https://www.image-net.org/).

We developed this solution based on the [official smp code](https://github.com/qubvel-org/segmentation_models.pytorch).

## 1.3. Hardware and environment

- PyTorch  2.1.2

- Python  3.10 (ubuntu22.04)

- Cuda  11.8

- GPU  A40(48GB) * 1

- CPU  15 vCPU AMD EPYC 7543 32-Core Processor


With the current pipeline settings, the training process took about 2 * 72 hours (with 1 NVIDIA A40 GPU (48 GB) available).


# 2. Solution Reproduction Steps

<font color=red>If you only want to reproduce my inference result, you only need to read section 2.1 and 2.6.</font>

<font color=red>If you want to reproduce both my training and inference result, you need to read from section 2.1 step by step.</font>


## 2.1. Environment Setup
Please, run the following command to install all needed libraries and packages.

In [ ]:
! pip install -r requirements.txt

## 2.2. Download and unzip data

Download the original competition data and unzip it.

Run the shell script in the next cell to download the original competition train data and unzip it. You need to modify the following variable in the below cell:
- ```SRC_TRAIN_DATA_ROOT```, <font color=red>represents the path to save the downloaded training data.</font>

In [ ]:
%%bash

SRC_TRAIN_DATA_ROOT="./data/train_data/"

wget -P $SRC_TRAIN_DATA_ROOT https://xeek-public-287031953319-eb80.s3.amazonaws.com/dark-side/dark-side-train-data-part1.zip
wget -P $SRC_TRAIN_DATA_ROOT https://xeek-public-287031953319-eb80.s3.amazonaws.com/dark-side/dark-side-train-data-part2.zip
wget -P $SRC_TRAIN_DATA_ROOT https://xeek-public-287031953319-eb80.s3.amazonaws.com/dark-side/dark-side-train-data-part3.zip
wget -P $SRC_TRAIN_DATA_ROOT https://xeek-public-287031953319-eb80.s3.amazonaws.com/dark-side/dark-side-train-data-part4.zip
wget -P $SRC_TRAIN_DATA_ROOT https://xeek-public-287031953319-eb80.s3.amazonaws.com/dark-side/dark-side-train-data-part5.zip
wget -P $SRC_TRAIN_DATA_ROOT https://xeek-public-287031953319-eb80.s3.amazonaws.com/dark-side/dark-side-train-data-part6.zip
wget -P $SRC_TRAIN_DATA_ROOT https://xeek-public-287031953319-eb80.s3.amazonaws.com/dark-side/dark-side-train-data-part7.zip
wget -P $SRC_TRAIN_DATA_ROOT https://xeek-public-287031953319-eb80.s3.amazonaws.com/dark-side/dark-side-train-data-part8.zip
wget -P $SRC_TRAIN_DATA_ROOT https://xeek-public-287031953319-eb80.s3.amazonaws.com/dark-side/dark-side-train-data-part9.zip
wget -P $SRC_TRAIN_DATA_ROOT https://xeek-public-287031953319-eb80.s3.amazonaws.com/dark-side/dark-side-train-data-part10.zip
wget -P $SRC_TRAIN_DATA_ROOT https://xeek-public-287031953319-eb80.s3.amazonaws.com/dark-side/dark-side-train-data-part11.zip
wget -P $SRC_TRAIN_DATA_ROOT https://xeek-public-287031953319-eb80.s3.amazonaws.com/dark-side/dark-side-train-data-part12.zip
wget -P $SRC_TRAIN_DATA_ROOT https://xeek-public-287031953319-eb80.s3.amazonaws.com/dark-side/dark-side-train-data-part13.zip
wget -P $SRC_TRAIN_DATA_ROOT https://xeek-public-287031953319-eb80.s3.amazonaws.com/dark-side/dark-side-train-data-part14.zip
wget -P $SRC_TRAIN_DATA_ROOT https://xeek-public-287031953319-eb80.s3.amazonaws.com/dark-side/dark-side-train-data-part15.zip
wget -P $SRC_TRAIN_DATA_ROOT https://xeek-public-287031953319-eb80.s3.amazonaws.com/dark-side/dark-side-train-data-part16.zip
wget -P $SRC_TRAIN_DATA_ROOT https://xeek-public-287031953319-eb80.s3.amazonaws.com/dark-side/dark-side-train-data-part17.zip
wget -P $SRC_TRAIN_DATA_ROOT https://xeek-public-287031953319-eb80.s3.amazonaws.com/dark-side/dark-side-train-data-part18.zip
wget -P $SRC_TRAIN_DATA_ROOT https://xeek-public-287031953319-eb80.s3.amazonaws.com/dark-side/dark-side-train-data-part19.zip
wget -P $SRC_TRAIN_DATA_ROOT https://xeek-public-287031953319-eb80.s3.amazonaws.com/dark-side/dark-side-train-data-part20.zip

for file in $(find $SRC_TRAIN_DATA_ROOT/*.zip -type f); do
    echo "$file is a file"
    unzip -q $file -d $SRC_TRAIN_DATA_ROOT
    rm $file
done

## 2.3. Preprocess training data

Because volume-based data normalization is time-consuming, the training data needs to be normalized to the range of 0-255 before training process, and in order to facilitate data reading, all data shapes are transposed to the same shape (1259, 300, 300) offline before training.

Run the following code to get the normalized data. You need to modify the following three variables in the below code:
1) ```SRC_TRAIN_DATA_ROOT```, represents the path of the original training data. This folder contains all the training data.
2) ```DST_TRAIN_DATA_ROOT```, represents the storage path of the normalized and reshaped training data.
3) ```PROCESS_THREAD_NUM```, represents the number of threads when multi-threading is performing data normalization. <font color=red>This variable can not be greater than multiprocessing.cpu_count().</font>

In [ ]:
import os
import numpy as np
from natsort import natsorted
import multiprocessing

SRC_TRAIN_DATA_ROOT = r"./data/train_data/"
DST_TRAIN_DATA_ROOT = r"./data/train_data_normed/"
PROCESS_THREAD_NUM = 1

os.makedirs(DST_TRAIN_DATA_ROOT, exist_ok=True)

def rescale_volume(seismic, low=0, high=100):
    """
    Rescaling 3D seismic volumes 0-255 range, clipping values between low and high percentiles
    """

    minval = np.percentile(seismic, low)
    maxval = np.percentile(seismic, high)

    seismic = np.clip(seismic, minval, maxval)
    seismic = ((seismic - minval) / (maxval - minval)) * 255

    return seismic


def process(test_id):
    files = natsorted(os.listdir(f"{SRC_TRAIN_DATA_ROOT}/{test_id}"))
    data = np.load(os.path.join(SRC_TRAIN_DATA_ROOT, test_id, files[1]), allow_pickle=True, mmap_mode="r+")
    label = np.load(os.path.join(SRC_TRAIN_DATA_ROOT, test_id,files[0]), allow_pickle=True, mmap_mode="r+")

    data = data.astype(np.float32)
    label = label.astype(np.float32)

    save_dir = os.path.join(DST_TRAIN_DATA_ROOT, test_id)

    os.makedirs(save_dir, exist_ok=True)
    data = rescale_volume(data)

    np.save(f"{save_dir}/{files[1]}", data)
    np.save(f"{save_dir}/{files[0]}", label)


with multiprocessing.Pool(processes = PROCESS_THREAD_NUM) as pool:
    test_id_s = natsorted(os.listdir(SRC_TRAIN_DATA_ROOT))
    print(test_id_s)
    result = pool.map(process, test_id_s)

When you run the above code, you should get a folder structure like this in directory ```./data/train_data_normed/```


![train data dir](images/train_data_dir.png "train data folder Structure")


## 2.4. Kfold training data

We used local 5-fold cross validation during the training process. In this section, we divide the training data into 5-folds to obtain training and validation data for each fold.

Run the following code to kfold the training data. You need to modify the following two variables in the below code:
1) ```SRC_TRAIN_DATA_ROOT```, represents the path of the original training data. This folder contains all the training data.
2) ```KFOLD_TXT_SAVE_ROOT```, represents the path to save the txt file of training and validation data in each fold, <font color=red>which will be used in the training section</font>

<font color=red>Because the generated 5-fold division txt file is already provided in path ```./data/train_txt/```, you can skip the following code and directly use the txt file in folder ```./data/train_txt/``` for training.</font>

In [ ]:
import os
import numpy as np
from sklearn.model_selection import KFold

SRC_TRAIN_DATA_ROOT = r"./data/train_data_normed/"
KFOLD_TXT_SAVE_ROOT = r"./train_txt/"

NUM_FOLD = 5
RANDOM_SEED=123
os.makedirs(KFOLD_TXT_SAVE_ROOT, exist_ok=True)

all_train_case = np.asarray(os.listdir(SRC_TRAIN_DATA_ROOT))
kf = KFold(n_splits=NUM_FOLD, random_state=RANDOM_SEED, shuffle=True)
for i, (train_index, valid_index) in enumerate(kf.split(all_train_case)):
    train_case = all_train_case[train_index]
    valid_case = all_train_case[valid_index]

    np.savetxt(f"{KFOLD_TXT_SAVE_ROOT}/train_f{i}.txt", train_case, fmt="%s")
    np.savetxt(f"{KFOLD_TXT_SAVE_ROOT}/val_f{i}.txt", valid_case, fmt="%s")

## 2.5. Pipeline Training Configuration

The training configuration file of fold0 is located in ```./src/configs/train_config_f0.py```. <font color=red>In order to ensure the success of the training, the following variables in this file need to be modified.</font>
1) ```model.pretrained_path```, represents the path of the pretrained model. The model will be automatically loaded before training starts. It should be changed to <font color=red><u>./pretrained_model/tf_efficientnetv2_s_21k-6337ad01.pth</u>.</font>
1) ```dataset.train.root_dir```, represents the root directory of the prenormed training data, which should contain subfolders for the 400 training image pairs. <font color=red>It should be changed to the ```DST_TRAIN_DATA_ROOT``` where the transposed data was saved in Section 2.3.</font>
2) ```dataset.train.txt_file```, represents the txt file used for training. To reproduce our results, you need to use the fold0 data divided in Section 2.4 for training. <font color=red>It should be changed to the path where the train_f0.txt file saved in section 2.4 is located.</font>
3) ```dataset.val.root_dir```,  represents the root directory of the transposed training data,  should be the same as ```datasets.train.root_dir```.
4) ```dataset.val.txt_file```, represents the txt file used for validation. <font color=red>It should be changed to the path where the val_f0.txt file saved in section 2.4 is located.</font>
6) ```train_cfg.save_path```, represents the path where the training results are saved.
6) ```train_cfg.batch_size```, represents the mini batchsize of each GPU during training. <font color=red> When I use the A40 (48GB) GPU locally, I can set this variable to a maximum of 24. If you are using an AWS SageMaker g5.12xlarge instance, you may need to lower the batch_size. But maybe you can only fully reproduce my training results by using the same batch size</font>





And the training configuration file of fold1 is located in ```./src/configs/train_config_f1.py```. <font color=red>In order to ensure the success of the training, the following variables in this file need to be modified.</font>
1) ```model.pretrained_path```, represents the path of the pretrained model. The model will be automatically loaded before training starts. It should be changed to <font color=red><u>./pretrained_model/tf_efficientnetv2_s_21k-6337ad01.pth</u>.</font>
1) ```dataset.train.root_dir```, represents the root directory of the prenormed training data, which should contain subfolders for the 400 training image pairs. <font color=red>It should be changed to the ```DST_TRAIN_DATA_ROOT``` where the transposed data was saved in Section 2.3.</font>
2) ```dataset.train.txt_file```, represents the txt file used for training. To reproduce our results, you need to use the fold1 data divided in Section 2.4 for training. <font color=red>It should be changed to the path where the train_f1.txt file saved in section 2.4 is located.</font>
3) ```dataset.val.root_dir```,  represents the root directory of the transposed training data,  should be the same as ```datasets.train.root_dir```.
4) ```dataset.val.txt_file```, represents the txt file used for validation. <font color=red>It should be changed to the path where the val_f1.txt file saved in section 2.4 is located.</font>
6) ```train_cfg.save_path```, represents the path where the training results are saved.
6) ```train_cfg.batch_size```, represents the mini batchsize of each GPU during training. <font color=red> When I use the A40 (48GB) GPU locally, I can set this variable to a maximum of 24. If you are using an AWS SageMaker g5.12xlarge instance, you may need to lower the batch_size. But maybe you can only fully reproduce my training results by using the same batch size</font>


## 2.6. Training step
<font color=red>You can skip this step if you only want to inference with my pre-trained model, which is provided in </font>```./my_checkpoints/*.pth```.

Otherwise, run the following command to trigger the model training script. 

In the following command, the meaning of each variable is as follows:
1) ```./src/train.py```, this is the main file of the training script and does not need to be modified.

NOTE:

1) Using the same data and training configuration as mine, you can reproduce my training results.

2) <font color=red>IF you do training process,  The training result will be saved to ```./experiments/exp_f0```  and ```./experiments/exp_f1``` directory. And you should select the model with the highest local validate score saved in the above two paths as your final two models, which can be used to reproduce my test results.</font>

3) I spend nearly 80 hours to finish training by 1 x A40 locally. The training time can be used as a reference for you

In [ ]:
! python ./src/train.py

## 2.7. Inference step
To inference the model and form a predictions for test dataset please follow the instructions below. 

Firstly, download the test data, decompress it, and organize the test data into the format described below.


![test_data_dir](images/test_data_dir.png)


Run the shell script in the next cell to download the original competition test data and unzip it. You need to modify the following variable in the below cell:
```SRC_TEST_DATA_ROOT```, <font color=red>represents the path to save the downloaded test data.</font>

In [ ]:
%%bash

SRC_TEST_DATA_ROOT="./data/test_data/"

wget -P $SRC_TEST_DATA_ROOT https://xeek-public-287031953319-eb80.s3.amazonaws.com/dark-side/dark-side-test-data-part-1.zip
wget -P $SRC_TEST_DATA_ROOT https://xeek-public-287031953319-eb80.s3.amazonaws.com/dark-side/dark-side-test-data-part-2.zip
wget -P $SRC_TEST_DATA_ROOT https://xeek-public-287031953319-eb80.s3.amazonaws.com/dark-side/dark-side-test-data-part-3.zip
wget -P $SRC_TEST_DATA_ROOT https://xeek-public-287031953319-eb80.s3.amazonaws.com/dark-side/dark-side-test-data-part-4.zip
wget -P $SRC_TEST_DATA_ROOT https://xeek-public-287031953319-eb80.s3.amazonaws.com/dark-side/dark-side-test-data-part-5.zip


for file in $(find $SRC_TEST_DATA_ROOT/*.zip -type f); do
    echo "$file is a file"
    unzip -q $file -d $SRC_TEST_DATA_ROOT
    rm $file
done

Secondly, you need to load the trained model and perform inference.

In the following command, the meaning of each variable is as follows:

1) ```./src/submit.py```, this is the main file of the inference script and does not need to be modified.

2) in ```./src/submit.py```, <font color=red><u>the following variables in the "main func" need to be modified.</font>
    - ```cfg_path```, represents the configuration file of the ensemble inference model. The default path is ```./src/configs/infer_config.py```, no modification is required
    - ```ckpt```, is a list and contains the best model trained using f0 data and f1 data respectively. By default, it’s ```["./my_checkpoints/effv2s_f0_cv09017_lb09415.pth", "./my_checkpoints/effv2s_f1_cv08901_lb09445.pth"]``` with this configuration, you can reproduce my results on public lb. You can also replace it with the best model you have trained.
    - ```test_data_root```,  represents the path of the raw test data. The data does not need to be normalized in advance, it will be automatically normalized during the inference process.By default, it’s "./data/test_data/"
    - ```submission_file```,  represents the path where the final generated submission file is located. By default, it’s ```"./final_submission.npz"```

<font color=red>NOTE:</font>
<font color=red>I provided three models in ```./src/my_checkpoints```, 1 model trained with f0 data, 2 models trained with f1 data, and the final model ensemble used the models trained with f0 and f1. Ensemble ```effv2s_f0_cv09017_lb09415.pth + effv2s_f1_iter316k_cv08822_lb0945.pth```, I can get my highest score on public lb, 0.946467. But I will choose ```effv2s_f0_cv09017_lb09415.pth + effv2s_f1_cv08901_lb09445.pth``` to get my final ensemble result because the local cv scores of these two models are higher.</font>


|  models   | public lb score  |
|  ----  | ----  |
| <font color=red>effv2s_f0_cv09017_lb09415.pth + effv2s_f1_cv08901_lb09445.pth</font> | <font color=red>0.946128</font> |
| <font color=red>effv2s_f0_cv09017_lb09415.pth + effv2s_f1_iter316k_cv08822_lb0945.pth</font>  | <font color=red>0.946467</font> |

Secondly, you need to load the trained model and perform inference.

In the following command, the meaning of each variable is as follows:

1) ```./src/submit.py```, this is the main file of the inference script and does not need to be modified.

2) in ```./src/submit.py```, <font color=red><u>the following variables in the "main func" need to be modified.</font>
    - ```cfg_path```, represents the configuration file of the ensemble inference model. The default path is ```./src/configs/infer_config.py```, no modification is required
    - ```ckpt```, is a list and contains the best model trained using f0 data and f1 data respectively. By default, it’s ```["./my_checkpoints/effv2s_f0_cv09017_lb09415.pth", "./my_checkpoints/effv2s_f1_cv08901_lb09445.pth"]``` with this configuration, you can reproduce my results on public lb. You can also replace it with the best model you have trained.
        
    - ```test_data_root```,  represents the path of the raw test data. The data does not need to be normalized in advance, it will be automatically normalized during the inference process.By default, it’s "./data/test_data/"
    - ```submission_file```,  represents the path where the final generated submission file is located. By default, it’s ```"./final_submission.npz"```

In [ ]:
! python ./src/submit.py

Now, you get the final submission file for scoring.